# Store alaska inventory as parquet format

In [3]:
# seisbench planetary computer environment
!mamba env create --quiet -f ../seisbench-testing/environment.yml

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK

done
Installing pip dependencies: ...working... done


## QuakeML

Reading quakeML is slow. We can extract relevant data from this file into a tablular format and save to a more cloud-friendly performant format like Parquet https://parquet.apache.org/documentation/latest/ 

In [18]:
import obspy
import obsplus
import pandas as pd

In [4]:
%%time

cat = obspy.core.event.read_events('../alaska_catalog/data_acquisition/catalog_css/2018_10/XO_2018_10.quakeml') #obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2020-11-19T01:12:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
print(cat)

/srv/conda/envs/seisbench/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2020-11-19T01:12:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


437 Event(s) in Catalog:
2018-10-01T06:02:23.795000Z | +57.940, -153.929 | 2.1 ml | manual
2018-10-01T08:35:07.702000Z | +57.139, -156.563 | 2.5 ml | manual
...
2018-10-31T11:05:22.076000Z | +56.208, -148.992 | 2.4 ml | manual
2018-10-31T20:03:47.517000Z | +55.858, -149.523 | 2.4 ml | manual
To see all events call 'print(CatalogObject.__str__(print_all=True))'
CPU times: user 48.9 s, sys: 447 ms, total: 49.4 s
Wall time: 49.4 s


## NOTE: use obsplus package to facilitate dataframe conversions

## Events

In [7]:
%%time

dfE = obsplus.events_to_df(cat)
dfE.head()

CPU times: user 5.08 s, sys: 88 ms, total: 5.17 s
Wall time: 5.16 s


,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,event_id,horizontal_uncertainty,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,stations
0,2018-10-01 06:02:23.795,57.9402,-153.9286,54300.0,2.1,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cl8u67g,NaN,...,0.83,97.0,76.0,300.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ACH, ANCK, AU22, AUE, AUJK, AUSB, AUW, AUWS, B..."
1,2018-10-01 08:35:07.702,57.1389,-156.5634,81400.0,2.5,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cla82sf,NaN,...,1.60,87.0,72.0,300.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ANCK, ANPK, BRSE, CHGN, CNP, EP15, EP22, EP23,..."
2,2018-10-01 09:42:17.906,58.2321,-155.1851,3400.0,1.4,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018clav17c,NaN,...,0.42,26.0,18.0,200.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ACH, ANCK, EP15, ET17, ET19, ET20, KABU, KAHG,..."
3,2018-10-01 09:55:56.347,55.8944,-149.8541,12000.0,2.8,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018claxwm6,NaN,...,0.89,55.0,45.0,25000.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"BAL, BARK, BERG, BRSE, CNP, CRQ, EYAK, GLI, HI..."
4,2018-10-01 14:10:56.076,55.6810,-149.2303,12200.0,2.7,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cldnfg8,NaN,...,0.88,61.0,43.0,25000.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"BRSE, CNP, FID, GHO, HIN, ILSW, KAKN, KD01, KD..."


In [11]:
dfE.iloc[0]

time                                             2018-10-01 06:02:23.795000
latitude                                                            57.9402
longitude                                                         -153.9286
depth                                                               54300.0
magnitude                                                               2.1
event_description                                                          
associated_phase_count                                                  0.0
azimuthal_gap                                                           NaN
event_id                     quakeml:earthquake.alaska.edu/event/018cl8u67g
horizontal_uncertainty                                                  NaN
local_magnitude                                                         2.1
moment_magnitude                                                        NaN
duration_magnitude                                                      NaN
magnitude_ty

In [8]:
# ArrowInvalid: Casting from timestamp[ns] to timestamp[us] would lose data: 1610761489008999936
# https://github.com/pandas-dev/pandas/issues/31572 
# df.to_parquet('XO_2018_10.parquet')
dfE.to_parquet('XO_2018_10_events.parquet', version='2.6')

In [9]:
# Round trip

df = pd.read_parquet('XO_2018_10_events.parquet')
df.head()

,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,event_id,horizontal_uncertainty,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,stations
0,2018-10-01 06:02:23.795,57.9402,-153.9286,54300.0,2.1,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cl8u67g,NaN,...,0.83,97.0,76.0,300.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ACH, ANCK, AU22, AUE, AUJK, AUSB, AUW, AUWS, B..."
1,2018-10-01 08:35:07.702,57.1389,-156.5634,81400.0,2.5,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cla82sf,NaN,...,1.60,87.0,72.0,300.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ANCK, ANPK, BRSE, CHGN, CNP, EP15, EP22, EP23,..."
2,2018-10-01 09:42:17.906,58.2321,-155.1851,3400.0,1.4,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018clav17c,NaN,...,0.42,26.0,18.0,200.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ACH, ANCK, EP15, ET17, ET19, ET20, KABU, KAHG,..."
3,2018-10-01 09:55:56.347,55.8944,-149.8541,12000.0,2.8,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018claxwm6,NaN,...,0.89,55.0,45.0,25000.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"BAL, BARK, BERG, BRSE, CNP, CRQ, EYAK, GLI, HI..."
4,2018-10-01 14:10:56.076,55.6810,-149.2303,12200.0,2.7,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cldnfg8,NaN,...,0.88,61.0,43.0,25000.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"BRSE, CNP, FID, GHO, HIN, ILSW, KAKN, KD01, KD..."


## Arrivals

In [15]:
%%time 

dfA = obsplus.arrivals_to_df(cat)
dfA

CPU times: user 8.63 s, sys: 7.97 ms, total: 8.64 s
Wall time: 8.64 s


,resource_id,seed_id,pick_id,phase,time_correction,azimuth,distance,takeoff_angle,time_residual,horizontal_slowness_residual,...,earth_model_id,creation_time,author,agency_id,network,station,location,channel,origin_id,origin_time
0,quakeml:earthquake.alaska.edu/arrival/137,XO.KD02.--.HHE,quakeml:earthquake.alaska.edu/pick/137,S,NaN,96.08,0.680,NaN,-0.072,NaN,...,,2020-11-12 20:08:35.813,"Alaska Earthquake Center, UAF/GI",AK,XO,KD02,--,HHE,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
1,quakeml:earthquake.alaska.edu/arrival/98,XO.KD05.--.HHZ,quakeml:earthquake.alaska.edu/pick/98,P,NaN,103.73,0.284,NaN,-0.533,NaN,...,,2020-10-02 22:32:18.450,"Alaska Earthquake Center, UAF/GI",AK,XO,KD05,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
2,quakeml:earthquake.alaska.edu/arrival/99,XO.KT06.--.HHZ,quakeml:earthquake.alaska.edu/pick/99,P,NaN,187.97,0.291,NaN,-0.252,NaN,...,,2020-11-13 01:05:19.635,"Alaska Earthquake Center, UAF/GI",AK,XO,KT06,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
3,quakeml:earthquake.alaska.edu/arrival/100,XO.KT07.--.HHZ,quakeml:earthquake.alaska.edu/pick/100,P,NaN,184.02,0.404,NaN,-0.166,NaN,...,,2020-10-02 22:32:18.450,"Alaska Earthquake Center, UAF/GI",AK,XO,KT07,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
4,quakeml:earthquake.alaska.edu/arrival/101,AK.R18K.--.BHZ,quakeml:earthquake.alaska.edu/pick/101,P,NaN,217.02,0.467,NaN,-0.377,NaN,...,,2018-10-05 17:57:13.748,"Alaska Earthquake Center, UAF/GI",AK,AK,R18K,--,BHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28346,quakeml:earthquake.alaska.edu/arrival/27951,XO.KD01.--.HHE,quakeml:earthquake.alaska.edu/pick/27951,S,NaN,319.33,2.394,NaN,-2.320,NaN,...,,2020-12-19 02:31:29.641,"Alaska Earthquake Center, UAF/GI",AK,XO,KD01,--,HHE,quakeml:earthquake.alaska.edu/origin/018dz3ax1b,2018-10-31 20:03:47.517
28347,quakeml:earthquake.alaska.edu/arrival/27952,XO.WD55.--.HH1,quakeml:earthquake.alaska.edu/pick/27952,S,NaN,269.35,2.328,NaN,1.813,NaN,...,,2020-12-19 02:33:08.591,"Alaska Earthquake Center, UAF/GI",AK,XO,WD55,--,HH1,quakeml:earthquake.alaska.edu/origin/018dz3ax1b,2018-10-31 20:03:47.517
28348,quakeml:earthquake.alaska.edu/arrival/27953,II.KDAK.10.BH1,quakeml:earthquake.alaska.edu/pick/27953,S,NaN,320.25,2.552,NaN,-2.620,NaN,...,,2020-12-19 02:31:37.285,"Alaska Earthquake Center, UAF/GI",AK,II,KDAK,10,BH1,quakeml:earthquake.alaska.edu/origin/018dz3ax1b,2018-10-31 20:03:47.517
28349,quakeml:earthquake.alaska.edu/arrival/27954,XO.KD02.--.HHN,quakeml:earthquake.alaska.edu/pick/27954,S,NaN,320.77,2.637,NaN,-1.081,NaN,...,,2020-12-19 02:33:30.545,"Alaska Earthquake Center, UAF/GI",AK,XO,KD02,--,HHN,quakeml:earthquake.alaska.edu/origin/018dz3ax1b,2018-10-31 20:03:47.517


In [12]:
dfA.iloc[0]

resource_id                           quakeml:earthquake.alaska.edu/arrival/137
seed_id                                                          XO.KD02.--.HHE
pick_id                                  quakeml:earthquake.alaska.edu/pick/137
phase                                                                         S
time_correction                                                             NaN
azimuth                                                                   96.08
distance                                                                   0.68
takeoff_angle                                                               NaN
time_residual                                                            -0.072
horizontal_slowness_residual                                                NaN
backazimuth_residual                                                        NaN
time_weight                                                               0.125
horizontal_slowness_weight              

In [17]:
dfA.to_parquet('XO_2018_10_arrivals.parquet', version='2.6')

In [19]:
# round trip
df = pd.read_parquet('XO_2018_10_arrivals.parquet')
df.head()

,resource_id,seed_id,pick_id,phase,time_correction,azimuth,distance,takeoff_angle,time_residual,horizontal_slowness_residual,...,earth_model_id,creation_time,author,agency_id,network,station,location,channel,origin_id,origin_time
0,quakeml:earthquake.alaska.edu/arrival/137,XO.KD02.--.HHE,quakeml:earthquake.alaska.edu/pick/137,S,NaN,96.08,0.680,NaN,-0.072,NaN,...,,2020-11-12 20:08:35.813,"Alaska Earthquake Center, UAF/GI",AK,XO,KD02,--,HHE,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
1,quakeml:earthquake.alaska.edu/arrival/98,XO.KD05.--.HHZ,quakeml:earthquake.alaska.edu/pick/98,P,NaN,103.73,0.284,NaN,-0.533,NaN,...,,2020-10-02 22:32:18.450,"Alaska Earthquake Center, UAF/GI",AK,XO,KD05,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
2,quakeml:earthquake.alaska.edu/arrival/99,XO.KT06.--.HHZ,quakeml:earthquake.alaska.edu/pick/99,P,NaN,187.97,0.291,NaN,-0.252,NaN,...,,2020-11-13 01:05:19.635,"Alaska Earthquake Center, UAF/GI",AK,XO,KT06,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
3,quakeml:earthquake.alaska.edu/arrival/100,XO.KT07.--.HHZ,quakeml:earthquake.alaska.edu/pick/100,P,NaN,184.02,0.404,NaN,-0.166,NaN,...,,2020-10-02 22:32:18.450,"Alaska Earthquake Center, UAF/GI",AK,XO,KT07,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
4,quakeml:earthquake.alaska.edu/arrival/101,AK.R18K.--.BHZ,quakeml:earthquake.alaska.edu/pick/101,P,NaN,217.02,0.467,NaN,-0.377,NaN,...,,2018-10-05 17:57:13.748,"Alaska Earthquake Center, UAF/GI",AK,AK,R18K,--,BHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795


## Picks

In [14]:
%%time
dfP = obsplus.picks_to_df(cat)
dfP

CPU times: user 2.98 s, sys: 1 µs, total: 2.98 s
Wall time: 2.98 s


,resource_id,time,seed_id,filter_id,method_id,horizontal_slowness,backazimuth,onset,phase_hint,polarity,...,agency_id,event_id,network,station,location,channel,uncertainty,lower_uncertainty,upper_uncertainty,confidence_level
0,quakeml:earthquake.alaska.edu/pick/137,2018-10-01 06:02:47.026,XO.KD02.--.HHE,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KD02,--,HHE,NaN,NaN,NaN,NaN
1,quakeml:earthquake.alaska.edu/pick/98,2018-10-01 06:02:32.200,XO.KD05.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KD05,--,HHZ,NaN,NaN,NaN,NaN
2,quakeml:earthquake.alaska.edu/pick/99,2018-10-01 06:02:32.539,XO.KT06.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KT06,--,HHZ,NaN,NaN,NaN,NaN
3,quakeml:earthquake.alaska.edu/pick/100,2018-10-01 06:02:33.620,XO.KT07.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KT07,--,HHZ,NaN,NaN,NaN,NaN
4,quakeml:earthquake.alaska.edu/pick/101,2018-10-01 06:02:34.048,AK.R18K.--.BHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,AK,R18K,--,BHZ,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28346,quakeml:earthquake.alaska.edu/pick/27951,2018-10-31 20:04:46.407,XO.KD01.--.HHE,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018dz3ax1b,XO,KD01,--,HHE,NaN,NaN,NaN,NaN
28347,quakeml:earthquake.alaska.edu/pick/27952,2018-10-31 20:04:48.557,XO.WD55.--.HH1,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018dz3ax1b,XO,WD55,--,HH1,NaN,NaN,NaN,NaN
28348,quakeml:earthquake.alaska.edu/pick/27953,2018-10-31 20:04:49.984,II.KDAK.10.BH1,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018dz3ax1b,II,KDAK,10,BH1,NaN,NaN,NaN,NaN
28349,quakeml:earthquake.alaska.edu/pick/27954,2018-10-31 20:04:53.576,XO.KD02.--.HHN,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018dz3ax1b,XO,KD02,--,HHN,NaN,NaN,NaN,NaN


In [16]:
dfP.iloc[0]

resource_id                    quakeml:earthquake.alaska.edu/pick/137
time                                       2018-10-01 06:02:47.026000
seed_id                                                XO.KD02.--.HHE
filter_id                                                            
method_id                                                            
horizontal_slowness                                               NaN
backazimuth                                                       NaN
onset                                                                
phase_hint                                                           
polarity                                                             
evaluation_mode                                                manual
event_time                                 2018-10-01 06:02:23.795000
evaluation_status                                                    
creation_time                              2020-11-12 20:08:35.813000
author              

In [20]:
dfP.to_parquet('XO_2018_10_picks.parquet', version='2.6')
df = pd.read_parquet('XO_2018_10_picks.parquet')
df.head()

,resource_id,time,seed_id,filter_id,method_id,horizontal_slowness,backazimuth,onset,phase_hint,polarity,...,agency_id,event_id,network,station,location,channel,uncertainty,lower_uncertainty,upper_uncertainty,confidence_level
0,quakeml:earthquake.alaska.edu/pick/137,2018-10-01 06:02:47.026,XO.KD02.--.HHE,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KD02,--,HHE,NaN,NaN,NaN,NaN
1,quakeml:earthquake.alaska.edu/pick/98,2018-10-01 06:02:32.200,XO.KD05.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KD05,--,HHZ,NaN,NaN,NaN,NaN
2,quakeml:earthquake.alaska.edu/pick/99,2018-10-01 06:02:32.539,XO.KT06.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KT06,--,HHZ,NaN,NaN,NaN,NaN
3,quakeml:earthquake.alaska.edu/pick/100,2018-10-01 06:02:33.620,XO.KT07.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KT07,--,HHZ,NaN,NaN,NaN,NaN
4,quakeml:earthquake.alaska.edu/pick/101,2018-10-01 06:02:34.048,AK.R18K.--.BHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,AK,R18K,--,BHZ,NaN,NaN,NaN,NaN


In [21]:
!ls -ltrh XO*

-rw-rw-r-- 1 jovyan users   30M Jan 19 00:46 XO_2018_10.quakeml
-rw-rw-r-- 1 jovyan users   85K Jan 19 16:46 XO_2018_10.parquet
-rw-r--r-- 1 jovyan users   85K Jan 19 18:34 XO_2018_10_events.parquet
-rw-r--r-- 1 jovyan users 1016K Jan 19 18:40 XO_2018_10_arrivals.parquet
-rw-r--r-- 1 jovyan users  772K Jan 19 18:41 XO_2018_10_picks.parquet


## Remote Read

In [23]:
%%time

dfP = pd.read_parquet('https://github.com/scottyhq/quakeflow-examples/raw/main/XO_2018_10_picks.parquet')
dfE = pd.read_parquet('https://github.com/scottyhq/quakeflow-examples/raw/main/XO_2018_10_events.parquet')
dfA = pd.read_parquet('https://github.com/scottyhq/quakeflow-examples/raw/main/XO_2018_10_arrivals.parquet')

CPU times: user 215 ms, sys: 28.4 ms, total: 244 ms
Wall time: 1.91 s
